# ***DISTRIBUCIÓN DE SEGMENTOS***

### DataFrames:

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [15]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Precio':'precio' }, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [16]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

### CONTEO ÚNICO DE CATEGORÍAS

In [17]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [19]:
conteo_total = {}
categorias_enlistadas = ['E1', 'E2', 'E3', 'D1', 'D2', 'D3', 'C1', 'C2', 'C3', 'B1', 'B2', 'B3', 'A1', 'A2', 'A3', 'S1', 'S2', 'S3', 'L1', 'L2', 'L3', 'L+', 'ELITE']
for df in dataframes_list:
    df.loc[:, 'categoria'] = df['precio'].apply(asignar_categoria)
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total, incluyendo categorías no presentes con conteo 0
    for categoria in categorias_enlistadas:
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo_actual.get(categoria, 0)

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\245307809.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\245307809.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\245307809.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,categoria,conteo_total
0,E1,87
1,E2,462
2,E3,297
3,D1,189
4,D2,121
5,D3,76
6,C1,86
7,C2,44
8,C3,38
9,B1,31


In [36]:
conteo_total = {}
categorias_enlistadas = ['E1', 'E2', 'E3', 'D1', 'D2', 'D3', 'C1', 'C2', 'C3', 'B1', 'B2', 'B3', 'A1', 'A2', 'A3', 'S1', 'S2', 'S3', 'L1', 'L2', 'L3', 'L+', 'ELITE']
for df in dataframes_list:
    df['categoria'] = df['precio'].apply(asignar_categoria)
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total, incluyendo categorías no presentes con conteo 0
    for categoria in categorias_enlistadas:
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo_actual.get(categoria, 0)

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
# Separar los números de las letras en otra columna, excepto para 'ELITE'
conteo_total_df['segmento'] = conteo_total_df['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])
conteo_total_df['clase'] = conteo_total_df['categoria'].apply(lambda x: '1' if x == 'ELITE' else x[1:])
conteo_total_df.drop(columns=['categoria'], inplace=True)
conteo_total_df

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\2204146326.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\2204146326.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3444\2204146326.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,conteo_total,segmento,clase
0,87,E,1
1,462,E,2
2,297,E,3
3,189,D,1
4,121,D,2
5,76,D,3
6,86,C,1
7,44,C,2
8,38,C,3
9,31,B,1


In [37]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_df.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         35
          2         21
          3         23
B         1         31
          2         59
          3         29
C         1         86
          2         44
          3         38
D         1        189
          2        121
          3         76
E         1         87
          2        462
          3        297
ELITE     1          0
L         +          0
          1          1
          2          2
          3          6
S         1         79
          2         15
          3         18
Name: conteo_total, dtype: int64

## Porcentajes totales de registros por segmento

In [38]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  1719


,segmento,total_por_segmento,porcentaje_del_total
0,A,79,4.595695
1,B,119,6.922629
2,C,168,9.773124
3,D,386,22.454916
4,E,846,49.214660
5,ELITE,0,0.000000
6,L,9,0.523560
7,S,112,6.515416


In [40]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Corregir el caso de 'ELITE' de clase '1' para que no salga NaN cuando es 0
segmentos_percent['porcentaje'] = segmentos_percent['porcentaje'].fillna(0)
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,E,1,10.283688
1,E,2,54.609929
2,E,3,35.106383
3,D,1,48.963731
4,D,2,31.347150
5,D,3,19.689119
6,C,1,51.190476
7,C,2,26.190476
8,C,3,22.619048
9,B,1,26.050420


Porcentaje de cada clase en relación al % que representa su segmento del total.

In [41]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,1,2.036067
1,A,2,1.221640
2,A,3,1.337987
3,B,1,1.803374
4,B,2,3.432228
5,B,3,1.687027
6,C,1,5.002909
7,C,2,2.559628
8,C,3,2.210588
9,D,1,10.994764


In [42]:
# Agrupar por 'segmento' y 'clase', y sumar los porcentajes
porcentajes_clase_agrupado = porcentajes_clase.groupby(['segmento', 'clase']).agg({'porcentaje_clase': 'sum'}).reset_index()

# Luego aplicar pivot
porcentajes_clase_ordenado = porcentajes_clase_agrupado.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,+,1,2,3
segmento,,,,
A,0.0,2.036067,1.221640,1.337987
B,0.0,1.803374,3.432228,1.687027
C,0.0,5.002909,2.559628,2.210588
D,0.0,10.994764,7.038976,4.421175
E,0.0,5.061082,26.876091,17.277487
ELITE,0.0,0.000000,0.000000,0.000000
L,0.0,0.058173,0.116347,0.349040
S,0.0,4.595695,0.872600,1.047120


## Gráfica

In [43]:
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S','L','ELITE']  # Orden deseado 'L'
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
clases = ['1', '2', '3']  # Clases en el orden deseado
# Asegúrate de que todos los segmentos mencionados estén en los datos con valor 0 si no existen
for segmento in segmentos_sorted:
    if segmento not in porcentajes_clase_ordenado.index:
        porcentajes_clase_ordenado.loc[segmento] = [0] * len(clases)

# Reordenar el DataFrame para seguir el orden deseado en `segmentos_sorted`
porcentajes_clase_ordenado = porcentajes_clase_ordenado.loc[segmentos_sorted]
colores = {
    '1': '#2962ff',
    '2': '#9500ff',
    '3': '#ff0059'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado[clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    yaxis=dict(
        title='Segmento', # Título del eje y        
    ),
    legend_title="Clase",
    barmode='stack',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

fig.show()

In [44]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_dist_segmentos', carpeta='assets/graficas')

